In [47]:
import numpy as np
import pandas as pd
import os

In [48]:
from sklearn import preprocessing

In [49]:
# pip install ipynb

In [84]:
def remove_uninformative_attrs(df):
    relevant_cols = ['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', \
                     'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', \
                     'duration_ms', 'time_signature', 'category']
    return df[relevant_cols]

In [113]:
def load_dataset(path='../data'):
    frames = []
    
    for p in os.listdir(path):
        df = pd.DataFrame(pd.read_csv(os.path.join(path, p)))
        df['category'] = p[2:-4]
        df = remove_uninformative_attrs(df)
        df['mode'] = [0 if mode == 'minor' else 1 for mode in df['mode']]
        df = pd.concat([df, pd.get_dummies(df['key'])], axis=1)
        df = df.drop('key', axis = 1)
        frames.append(df)
        
    df = pd.concat(frames)
    df = df.dropna()
    return df

In [114]:
def scale_data(x_train, x_test):
    scaler = preprocessing.StandardScaler()
    scaler.fit(x_train)
    x_train_transformed = scaler.transform(x_train)
    x_test_transformed = scaler.transform(x_test)
    return (x_train_transformed, x_test_transformed)

In [115]:
df = load_dataset()
df.head()

,danceability,energy,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,...,B,C,C#,D,D#,E,F,F#,G,G#
0,0.901,0.522,-8.304,1,0.3300,0.01140,0.00000,0.259,0.329,104.053,...,0,0,0,1,0,0,0,0,0,0
1,0.834,0.730,-3.714,1,0.2220,0.00513,0.00000,0.124,0.446,155.008,...,0,0,0,0,0,0,0,0,0,1
3,0.876,0.786,-4.884,0,0.3060,0.06270,0.00000,0.127,0.575,153.068,...,0,0,0,0,0,0,0,0,0,0
4,0.900,0.653,-6.962,0,0.2890,0.09450,0.00000,0.528,0.399,112.503,...,0,0,1,0,0,0,0,0,0,0
5,0.738,0.629,-4.882,0,0.0495,0.26800,0.00114,0.163,0.319,146.067,...,0,0,0,0,0,0,0,1,0,0
